In [1]:
PPMC_ORDER = 4
ID = 0
TEXT = 9
PREDICTION = 6

fieldnames = ["Id", "ProductId", "UserId", "ProfileName", "HelpfulnessNumerator",
            "HelpfulnessDenominator", "Prediction", "Time", "Summary", "Text"]

import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import csv
from PPMC import PPMC

#text = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."

trainRDD = sc.textFile("train_sin_repeticiones.csv", minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))
trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.sample(False, 0.01)

#se podria hacer un map antes que limpie el texto
#comprimo todo el train con ppmc
trainRDD = trainRDD.map(lambda line: (line[PREDICTION], (PPMC(PPMC_ORDER), line[TEXT])))

trainRDD = trainRDD.map(lambda line: (line[0], (line[1][0], 1, line[1][0].compress(line[1][1]))))

In [ ]:
l=[0]

def foo(value1, value2):
    print l[0]
    l[0]+=1
    return (value1[0]+value2[0], value1[1]+value2[1])

test_prediction = open("test_prediction", "w")
writer = csv.DictWriter(test_prediction, fieldnames=fieldnames)
writer.writeheader()

with open("test.csv", "r") as test:
    reader = csv.DictReader(test)
    for row in reader:
        text = row["Text"]
        #comprimo el texto de prueba con los modelos de todo el rdd
        testRDD = trainRDD.map(lambda line: (line[0], (len(line[1][0].compress(text)), line[1][1])))
        testRDD = testRDD.reduceByKey(lambda value1, value2: foo(value1, value2))
        testRDD = testRDD.reduce(lambda line1, line2: line1 if line1[1][0]/line1[1][1] < line2[1][0]/line2[1][1] else line2)
        row["Prediction"] = testRDD[0]
        writer.writerow(row)